# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 05**: Data pipeline with Neo4j

**Date**: October 2nd 2025

**Student Name**:

**Professor**: Pablo Camarillo Ramirez

# Dataset description

# Data ingestion

In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.neo4j:neo4j-connector-apache-spark_2.13:5.3.10_for_spark_3") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.neo4j#neo4j-connector-apache-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-699bdc11-c7d0-4bc4-b708-098d59a6bcb1;1.0
	confs: [default]
	found org.neo4j#neo4j-connector-apache-spark_2.13;5.3.10_for_spark_3 in central
	found org.neo4j#neo4j-connector-apache-spark_2.13_common;5.3.10_for_spark_3 in central
	found org.neo4j#caniuse-core;1.3.0 in central
	found org.neo4j#caniuse-api;1.3.0 in central
	found org.jetbrains.kotlin#kotlin-stdlib;2.1.20 in central
	found org.jetbrains#annotations;13.0 in central
	found org.neo4j#caniuse-neo4j-detection;1.3.0 in central
	found org.neo4j.driver#neo4j-java-driver-slim;4.4.21 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found io.netty#netty-handler;4.1.

In [4]:
# Build schema
# Import your module
from pcamarillor.spark_utils import SparkUtils

pokemon_schema_columns = [
    ("#", "int"),
    ("Name", "string"),
    ("Type 1", "string"),
    ("Type 2", "string"),
    ("Total", "int"),
    ("HP", "int"),
    ("Attack", "int"),
    ("Defense", "int"),
    ("Sp. Atk", "int"),
    ("Sp. Def", "int"),
    ("Speed", "int")
]

pokemon_schema = SparkUtils.generate_schema(pokemon_schema_columns)

pokemon_df = spark.read \
    .option("header", "true") \
    .schema(pokemon_schema) \
    .csv("/opt/spark/work-dir/data/pokemon/Pokemon.csv")

pokemon_df.show(5)

[Stage 0:>                                                          (0 + 1) / 1]

+---+--------------------+------+------+-----+---+------+-------+-------+-------+-----+
|  #|                Name|Type 1|Type 2|Total| HP|Attack|Defense|Sp. Atk|Sp. Def|Speed|
+---+--------------------+------+------+-----+---+------+-------+-------+-------+-----+
|  1|           Bulbasaur| Grass|Poison|  318| 45|    49|     49|     65|     65|   45|
|  2|             Ivysaur| Grass|Poison|  405| 60|    62|     63|     80|     80|   60|
|  3|            Venusaur| Grass|Poison|  525| 80|    82|     83|    100|    100|   80|
|  3|VenusaurMega Venu...| Grass|Poison|  625| 80|   100|    123|    122|    120|   80|
|  4|          Charmander|  Fire|  NULL|  309| 39|    52|     43|     60|     50|   65|
+---+--------------------+------+------+-----+---+------+-------+-------+-------+-----+
only showing top 5 rows


# Transformations

In [13]:
# Add the code for your transformations to create nodes and edges DataFrames HERE
from pyspark.sql.functions import col

pokemon_nodes = pokemon_df.select(
    col("#").alias("id"),
    col("Name").alias("name"),
    col("Total"),
    col("HP"),
    col("Attack"),
    col("Defense"),
    col("`Sp. Atk`").alias("sp_atk"),
    col("`Sp. Def`").alias("sp_def"),
    col("Speed")
)
pokemon_nodes.show()

type_nodes = pokemon_df.select(
    col("Type 1").alias("name")
).union(
    pokemon_df.select(col("Type 2").alias("name"))
).dropna().dropDuplicates(["name"])
type_nodes.show()

edges_type1 = pokemon_df.select(
    col("Name").alias("src"),
    col("Type 1").alias("dst")
)
edges_type1.show()

edges_type2 = pokemon_df.select(
    col("Name").alias("src"),
    col("Type 2").alias("dst")
).filter(col("Type 2").isNotNull())
edges_type2.show()


+---+--------------------+-----+---+------+-------+------+------+-----+
| id|                name|Total| HP|Attack|Defense|sp_atk|sp_def|Speed|
+---+--------------------+-----+---+------+-------+------+------+-----+
|  1|           Bulbasaur|  318| 45|    49|     49|    65|    65|   45|
|  2|             Ivysaur|  405| 60|    62|     63|    80|    80|   60|
|  3|            Venusaur|  525| 80|    82|     83|   100|   100|   80|
|  3|VenusaurMega Venu...|  625| 80|   100|    123|   122|   120|   80|
|  4|          Charmander|  309| 39|    52|     43|    60|    50|   65|
|  5|          Charmeleon|  405| 58|    64|     58|    80|    65|   80|
|  6|           Charizard|  534| 78|    84|     78|   109|    85|  100|
|  6|CharizardMega Cha...|  634| 78|   130|    111|   130|    85|  100|
|  6|CharizardMega Cha...|  634| 78|   104|     78|   159|   115|  100|
|  7|            Squirtle|  314| 44|    48|     65|    50|    64|   43|
|  8|           Wartortle|  405| 59|    63|     80|    65|    80

# Writing Data in Neo4j

In [14]:
# Add the code to write a graph from PySpark's DataFrames to Neo4j
neo4j_url = "bolt://neo4j-iteso:7687"
neo4j_user = "neo4j"
neo4j_passwd = "neo4j@1234"

pokemon_nodes.write \
    .format("org.neo4j.spark.DataSource") \
    .mode("Overwrite") \
    .option("url", neo4j_url) \
    .option("authentication.basic.username", neo4j_user) \
    .option("authentication.basic.password", neo4j_passwd) \
    .option("labels", ":Pokemon") \
    .option("node.keys", "id") \
    .save()
print(f"{pokemon_nodes.count()} pokemon nodes wrote in Neo4j")

type_nodes.write \
    .format("org.neo4j.spark.DataSource") \
    .mode("Overwrite") \
    .option("url", neo4j_url) \
    .option("authentication.basic.username", neo4j_user) \
    .option("authentication.basic.password", neo4j_passwd) \
    .option("labels", ":Type") \
    .option("node.keys", "name") \
    .save()
print(f"{type_nodes.count()} type nodes wrote in Neo4j")

type_edges.write \
    .format("org.neo4j.spark.DataSource") \
    .mode("Overwrite") \
    .option("url", neo4j_url) \
    .option("authentication.basic.username", neo4j_user) \
    .option("authentication.basic.password", neo4j_passwd) \
    .option("relationship", "HAS_TYPE") \
    .option("relationship.save.strategy", "keys") \
    .option("relationship.source.labels", ":Pokemon") \
    .option("relationship.source.save.mode", "match") \
    .option("relationship.source.node.keys", "src:name") \
    .option("relationship.target.labels", ":Type") \
    .option("relationship.target.save.mode", "match") \
    .option("relationship.target.node.keys", "dst:name") \
    .save()
print(f"{type_edges.count()} type edges wrote in Neo4j")


800 pokemon nodes wrote in Neo4j
18 type nodes wrote in Neo4j


1214 type edges wrote in Neo4j


# Read and Query Graphs with PySpark

In [16]:
# Add the code to read a data frame from Neo4J and run a simple query to verify

pokemon_types_df = spark.read \
    .format("org.neo4j.spark.DataSource") \
    .option("url", neo4j_url) \
    .option("authentication.basic.username", neo4j_user) \
    .option("authentication.basic.password", neo4j_passwd) \
    .option("query",
            """
            MATCH (p:Pokemon)-[r:HAS_TYPE]->(t:Type)
            RETURN p.name AS pokemon, t.name AS type
            ORDER BY p.name
            """) \
    .load()

pokemon_types_df.show()


+--------------------+-------+
|             pokemon|   type|
+--------------------+-------+
|AbomasnowMega Abo...|    Ice|
|AbomasnowMega Abo...|  Grass|
|                Abra|Psychic|
|     AbsolMega Absol|   Dark|
|            Accelgor|    Bug|
|AegislashShield F...|  Ghost|
|AegislashShield F...|  Steel|
|AerodactylMega Ae...| Flying|
|AerodactylMega Ae...|   Rock|
|   AggronMega Aggron|  Steel|
|               Aipom| Normal|
|AlakazamMega Alak...|Psychic|
|           Alomomola|  Water|
| AltariaMega Altaria|  Fairy|
| AltariaMega Altaria| Dragon|
|              Amaura|    Ice|
|              Amaura|   Rock|
|             Ambipom| Normal|
|           Amoonguss| Poison|
|           Amoonguss|  Grass|
+--------------------+-------+
only showing top 20 rows


In [ ]:
sc.stop()